# Carter frame in Kerr spacetime

This Jupyter/SageMath notebook is relative to the lectures
[Geometry and physics of black holes](https://luth.obspm.fr/~luthier/gourgoulhon/bh16/).

The corresponding tools have been developed within the  [SageManifolds](https://sagemanifolds.obspm.fr) project.

*NB:* a version of [SageMath](https://www.sagemath.org/) at least equal to 8.2 is required to run this notebook:

In [1]:
version()

'SageMath version 9.2.beta13, Release Date: 2020-09-21'

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [2]:
%display latex

We ask for running tensor computations in parallel on 8 threads:

In [3]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the part of it covered by Boyer-Lindquist coordinates) as a 4-dimensional Lorentzian manifold $\mathcal{M}$:

In [4]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


We then introduce the standard <strong>Boyer-Lindquist coordinates</strong> as a chart `BL` (for *Boyer-Lindquist*) on $\mathcal{M}$, via the method `chart()`, the argument of which is a string
(delimited by `r"..."` because of the backslash symbols) expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [5]:
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") 
print(BL); BL

Chart (M, (t, r, th, ph))


Chart (M, (t, r, th, ph))

In [6]:
BL[0], BL[1]

(t, r)

<h2>Metric tensor</h2>

<p>The 2 parameters $m$ and $a$ of the Kerr spacetime are declared as symbolic variables:</p>

In [7]:
var('m, a', domain='real')

(m, a)

We get the (yet undefined) spacetime metric:

In [8]:
g = M.metric()

<p>The metric is set by its components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:</p>

In [9]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt*dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph*dph

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [10]:
g[:]

[                                  2*m*r/(a^2*cos(th)^2 + r^2) - 1                                                                 0                                                                 0                          -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)]
[                                                                0                         (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2)                                                                 0                                                                 0]
[                                                                0                                                                 0                                               a^2*cos(th)^2 + r^2                                                                 0]
[                         -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)                                                                 0                                                                 0 (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2]

<p>The list of the non-vanishing components:</p>

In [11]:
g.display_comp()

g_t,t = 2*m*r/(a^2*cos(th)^2 + r^2) - 1 
g_t,ph = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_r,r = (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) 
g_th,th = a^2*cos(th)^2 + r^2 
g_ph,t = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_ph,ph = (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2

## Carter frame
The default vector frame on the spacetime manifold is the coordinate basis associated with Boyer-Lindquist coordinates:

In [12]:
M.default_frame() is BL.frame()

True

In [13]:
BL.frame()

Coordinate frame (M, (d/dt,d/dr,d/dth,d/dph))

The 4-velocity of the Carter obsever is defined by its components w.r.t. the Boyer-Lindquist frame:

In [14]:
rho = sqrt(rho2)
e0 = M.vector_field([(r^2 + a^2)/(rho*sqrt(Delta)), 0, 0, a/(rho*sqrt(Delta))],
                    name='eps0', latex_name=r'\varepsilon_0')
e0.display()

eps0 = (a^2 + r^2)/(sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)) d/dt + a/(sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)) d/dph

We check that is a unit timelike vector:

In [15]:
g(e0, e0).expr()

-1

and that it is future-directed, by computing its scalar product with the global null vector $k$, which generates the ingoing principal null geodesics:

In [16]:
k = M.vector_field([(r^2 + a^2)/Delta, -1, 0, a/Delta],
                   name='k')
k.display()

k = (a^2 + r^2)/(a^2 - 2*m*r + r^2) d/dt - d/dr + a/(a^2 - 2*m*r + r^2) d/dph

In [17]:
g(k, e0).expr()

-sqrt(a^2*cos(th)^2 + r^2)/sqrt(a^2 - 2*m*r + r^2)

The spacelike vectors of the Carter frame:

In [18]:
e1 = M.vector_field([0, sqrt(Delta)/rho, 0, 0],
                    name='eps1', latex_name=r'\varepsilon_1')
e1.display()

eps1 = sqrt(a^2 - 2*m*r + r^2)/sqrt(a^2*cos(th)^2 + r^2) d/dr

In [19]:
e2 = M.vector_field([0, 0, 1/rho, 0],
                    name='eps2', latex_name=r'\varepsilon_2')
e2.display()

eps2 = 1/sqrt(a^2*cos(th)^2 + r^2) d/dth

In [20]:
e3 = M.vector_field([a*sin(th)/rho, 0, 0, 1/(rho*sin(th))],
                    name='eps3', latex_name=r'\varepsilon_3')
e3.display()

eps3 = a*sin(th)/sqrt(a^2*cos(th)^2 + r^2) d/dt + 1/(sqrt(a^2*cos(th)^2 + r^2)*sin(th)) d/dph

In [21]:
CF = M.vector_frame('eps', [e0, e1, e2, e3], latex_symbol=r'\varepsilon')
CF

Vector frame (M, (eps_0,eps_1,eps_2,eps_3))

In [22]:
for v in CF:
    show(v.display())

eps_0 = (a^2 + r^2)/(sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)) d/dt + a/(sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)) d/dph

eps_1 = sqrt(a^2 - 2*m*r + r^2)/sqrt(a^2*cos(th)^2 + r^2) d/dr

eps_2 = 1/sqrt(a^2*cos(th)^2 + r^2) d/dth

eps_3 = a*sin(th)/sqrt(a^2*cos(th)^2 + r^2) d/dt + 1/(sqrt(a^2*cos(th)^2 + r^2)*sin(th)) d/dph

Check that the Carter frame is orthonormal:

In [23]:
matrix([[g(u, v).expr() for v in CF] for u in CF])  

[-1  0  0  0]
[ 0  1  0  0]
[ 0  0  1  0]
[ 0  0  0  1]

### The Carter coframe

In [24]:
for f in CF.coframe():
    show(f.display())

eps^0 = sqrt(a^2 - 2*m*r + r^2)/sqrt(a^2*cos(th)^2 + r^2) dt - sqrt(a^2 - 2*m*r + r^2)*a*sin(th)^2/sqrt(a^2*cos(th)^2 + r^2) dph

eps^1 = sqrt(a^2*cos(th)^2 + r^2)/sqrt(a^2 - 2*m*r + r^2) dr

eps^2 = sqrt(a^2*cos(th)^2 + r^2) dth

eps^3 = -a*sin(th)/sqrt(a^2*cos(th)^2 + r^2) dt + (a^2 + r^2)*sin(th)/sqrt(a^2*cos(th)^2 + r^2) dph

## 4-acceleration of the Carter observer

In [25]:
nabla = g.connection()
print(nabla)

Levi-Civita connection nabla_g associated with the Lorentzian metric g on the 4-dimensional Lorentzian manifold M


In [26]:
A = nabla(e0).contract(e0)  # use A to avoid any confusion with the Kerr parameter a
A.set_name('a')
print(A)

Vector field a on the 4-dimensional Lorentzian manifold M


In [27]:
A.apply_map(factor)  # factor the components for a better display
A.display()

a = (a^2*m*sin(th)^2 - a^2*r*sin(th)^2 - a^2*m + m*r^2)/(a^2*cos(th)^2 + r^2)^2 d/dr - a^2*cos(th)*sin(th)/(a^2*cos(th)^2 + r^2)^2 d/dth

As a check, we note that the above formula agrees with that given by Eqs. (90)-(91) of 
[O. Semerak, Gen. Relat. Grav. **25**, 1041 (1993)](https://doi.org/10.1007/BF00763554).

Display in the Carter frame:

In [28]:
A.apply_map(factor, frame=CF, keep_other_components=True)
A.display(CF)

a = (a^2*m*sin(th)^2 - a^2*r*sin(th)^2 - a^2*m + m*r^2)/((a^2*cos(th)^2 + r^2)^(3/2)*sqrt(a^2 - 2*m*r + r^2)) eps_1 - a^2*cos(th)*sin(th)/(a^2*cos(th)^2 + r^2)^(3/2) eps_2

## 4-rotation of the ZAMO frame

We define the rotation operator as
$$ \Omega_{\rm rot}(v) = \nabla_{\varepsilon_0} v - \Omega_{\rm FW}(v)$$
where
$$\Omega_{\rm FW}(v) := (a\cdot v) \varepsilon_0 - (\varepsilon_0\cdot v) a$$
is the Fermi-Walker operator:

In [29]:
def rotation_operator(v):
    return nabla(v).contract(e0) - g(A, v)*e0 + g(e0, v)*A

Some check:

In [30]:
rotation_operator(e0) == 0

True

Let us evaluate $\Omega_{\rm rot}(\varepsilon_1)$:

In [31]:
De1 = rotation_operator(e1)
De1.apply_map(factor)
De1.display()

a^2*r*sin(th)^2/(a^2*cos(th)^2 + r^2)^2 d/dt + a*r/(a^2*cos(th)^2 + r^2)^2 d/dph

In [32]:
De1.apply_map(factor, frame=CF, keep_other_components=True)
De1.display(CF)

a*r*sin(th)/(a^2*cos(th)^2 + r^2)^(3/2) eps_3

Let us now evaluate $\Omega_{\rm rot}(\varepsilon_2)$:

In [33]:
De2 = rotation_operator(e2)
De2.apply_map(factor)
De2.display()

sqrt(a^2 - 2*m*r + r^2)*a^2*cos(th)*sin(th)/(a^2*cos(th)^2 + r^2)^2 d/dt + sqrt(a^2 - 2*m*r + r^2)*a*cos(th)/((a^2*cos(th)^2 + r^2)^2*sin(th)) d/dph

In [34]:
De2.apply_map(factor, frame=CF, keep_other_components=True)
De2.display(CF)

-(a^2*sin(th)^2 - a^2 - r^2)*sqrt(a^2 - 2*m*r + r^2)*a*cos(th)/(a^2*cos(th)^2 + r^2)^(5/2) eps_3

and finally $\Omega_{\rm rot}(e_3)$:

In [35]:
De3 = rotation_operator(e3)
De3.apply_map(factor)
De3.display()

-sqrt(a^2 - 2*m*r + r^2)*a*r*sin(th)/(a^2*cos(th)^2 + r^2)^2 d/dr + (a^2*sin(th)^2 - a^2 - r^2)*sqrt(a^2 - 2*m*r + r^2)*a*cos(th)/(a^2*cos(th)^2 + r^2)^3 d/dth

Let us enforce further trigonometric simplification: 

In [36]:
De3.apply_map(lambda x: x.trig_reduce())
De3.apply_map(lambda x: x.simplify_trig())
De3.apply_map(factor)
De3.display()

-sqrt(a^2 - 2*m*r + r^2)*a*r*sin(th)/(a^2*cos(th)^2 + r^2)^2 d/dr - sqrt(a^2 - 2*m*r + r^2)*a*cos(th)/(a^2*cos(th)^2 + r^2)^2 d/dth

In [37]:
De3.apply_map(factor, frame=CF, keep_other_components=True)
De3.display(CF)

-a*r*sin(th)/(a^2*cos(th)^2 + r^2)^(3/2) eps_1 - sqrt(a^2 - 2*m*r + r^2)*a*cos(th)/(a^2*cos(th)^2 + r^2)^(3/2) eps_2

In the orthonormal frame $(\varepsilon_1, \varepsilon_2, \varepsilon_3)$ of the Carter observer's rest space, the matrix of the operator $\Omega_{\rm rot}$ must be of the type 
$$
    \left( \begin{array}{ccc}
    0 & - \omega^3 & \omega^3 \\
    \omega^3 & 0 & -\omega^1 \\
    - \omega^2 & \omega^1 & 0
    \end{array} \right), 
$$
where the $\omega^i$'s are the components on the 4-rotation vector:
$$
    \omega = \omega^i \varepsilon_i
$$

From the above expressions of the $\Omega_{\rm rot}(\varepsilon_i)$'s, we get immediately $\omega^3 = 0$ and we check that

In [38]:
De1[CF, 3] == - De3[CF, 1]

True

In [39]:
De2[CF, 3] == - De3[CF, 2]

True

Therefore, we get the 4-rotation vector $\omega$ as

In [40]:
omega = - De3[CF, 2]*e1 + De3[CF, 1]*e2
omega.set_name('omega', latex_name=r'\omega')
omega.apply_map(factor)
omega.apply_map(factor, frame=CF, keep_other_components=True)
omega.display(CF)

omega = sqrt(a^2 - 2*m*r + r^2)*a*cos(th)/(a^2*cos(th)^2 + r^2)^(3/2) eps_1 - a*r*sin(th)/(a^2*cos(th)^2 + r^2)^(3/2) eps_2

In [41]:
omega.display()

omega = (a^2 - 2*m*r + r^2)*a*cos(th)/(a^2*cos(th)^2 + r^2)^2 d/dr - a*r*sin(th)/(a^2*cos(th)^2 + r^2)^2 d/dth

As a check, we note that the above formula agrees with that given by Eq. (93) of 
[O. Semerak, Gen. Relat. Grav. **25**, 1041 (1993)](https://doi.org/10.1007/BF00763554).